In [66]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import QuantLib as ql

from functions import *

In [67]:
def summary_table_frn(input_dictionary):

    start_dates = input_dictionary["start_dates"]
    pay_dates = input_dictionary["pay_dates"]
    year_fracs = input_dictionary["year_fracs"]
    payment_times = input_dictionary["payment_times"]
    discount_curve = input_dictionary["discount_curve"]
    cash_flows = input_dictionary["cash_flows"]
    discounted_cf = input_dictionary["discounted_cf"]
    spread = input_dictionary["spread"]
    coupon_rates = input_dictionary["coupon_rates_no_spread"]
    
    summary_dic = {
            "Start Date": start_dates,
            "Pay Date": pay_dates,
            "Year Frac": year_fracs,
            "Pay Time": payment_times,
            "Discount Curve": discount_curve,
            "Spread (bps)": spread,       
            "Coupon Rates (No Spread)": coupon_rates,
            "Cash Flows": cash_flows,
            "Discounted Cash Flows": discounted_cf,
    }

    summary = pd.DataFrame(summary_dic,
                            index = [i for i in range(1, len(start_dates)+1)] )
    
    return summary

In [68]:
def price_floating_rate_note(settlement_date, maturity_date, discount_curve, calendar, tenor, current_coupon_rate, spread, notional, day_count, day_rolling, schedule):

    scheduling_information = set_scheduling_information(schedule,
                                                        maturity_date,
                                                        day_count,
                                                        tenor,
                                                        settlement_date,
                                                        calendar,
                                                        day_rolling)

    accrual_period = scheduling_information["accrual_period"] 
    start_dates = scheduling_information["start_dates"]
    pay_dates = scheduling_information["pay_dates"]
    payment_times = scheduling_information["payment_times"]  
    accrued_interest = accrual_period*notional*current_coupon_rate

    payment_times = np.array(payment_times)
    
    year_fracs = scheduling_information["year_fracs"]  
    year_fracs = np.array(year_fracs)

    discount_curve = set_discount_curve(discount_curve, payment_times)
    spread_percentage = spread/10000

    coupon_rates = []

    for index in range(len(pay_dates)):
        
        tau_forward = year_fracs[index]

        if index == 0:
            coupon_rate = current_coupon_rate-spread_percentage
    
        else:
            df_start = discount_curve[index-1]
            df_end = discount_curve[index]
            
            forward_rate = 1/tau_forward * (df_start/df_end-1)

            coupon_rate = forward_rate
            
        coupon_rates.append(coupon_rate)

    
    coupon_rates = np.array(coupon_rates) + spread_percentage
    coupons = coupon_rates*notional*year_fracs

    cash_flows = coupons
    cash_flows[-1] += notional

    discounted_cash_flows = cash_flows*discount_curve

    dirty_price = np.sum(discounted_cash_flows)
    clean_price = dirty_price - accrued_interest
    
    ### Prepare Table

    input_dictionary = {
        "start_dates": start_dates,
        "pay_dates": pay_dates,
        "year_fracs": year_fracs,
        "payment_times": payment_times,
        "discount_curve": discount_curve,
        "cash_flows": cash_flows,
        "discounted_cf": discounted_cash_flows,
        "spread": spread,
        "coupon_rates_no_spread": coupon_rates-spread_percentage
    }

    summary_table = summary_table_frn(input_dictionary)

    return {"Dirty Price": dirty_price, "Accrued Interest": accrued_interest, "Clean Price": clean_price, "Summary Table": summary_table}    

In [70]:
notional = 100
current_coupon = 0.0213
spread = 200

calendar = ql.TARGET()

settlement_date = ql.Date(12,11,2025)
maturity_date = ql.Date(18,6,2027)

tenor = ql.Period(1,ql.Months)
day_count = ql.Actual360()
day_rolling = ql.ModifiedFollowing
termination_day_rolling = ql.ModifiedFollowing
end_of_month = False
date_generation = ql.DateGeneration.Backward

discount_curve = pd.read_excel("tsData2025.xlsx", index_col= 0)

schedule = ql.Schedule( settlement_date,
                        maturity_date,
                        tenor,
                        calendar,
                        day_rolling,
                        termination_day_rolling,
                        date_generation,
                        end_of_month)   

In [71]:
price = price_floating_rate_note(
    settlement_date,
    maturity_date,
    discount_curve,
    calendar,
    tenor,
    current_coupon,
    spread,
    notional,
    day_count,
    day_rolling,
    schedule
)

In [73]:
price["Dirty Price"]

103.36679081080253

In [74]:
price["Clean Price"]

103.2307074774692

In [72]:
price["Summary Table"]

,Start Date,Pay Date,Year Frac,Pay Time,Discount Curve,Spread (bps),Coupon Rates (No Spread),Cash Flows,Discounted Cash Flows
1,"October 20th, 2025","November 18th, 2025",0.080556,0.016438,0.999976,200,0.001300,0.171583,0.171579
2,"November 18th, 2025","December 18th, 2025",0.083333,0.098630,0.999842,200,0.001607,0.180060,0.180031
3,"December 18th, 2025","January 19th, 2026",0.088889,0.186301,0.999676,200,0.001872,0.194416,0.194353
4,"January 19th, 2026","February 18th, 2026",0.083333,0.268493,0.999498,200,0.002136,0.184470,0.184378
5,"February 18th, 2026","March 18th, 2026",0.077778,0.345205,0.999312,200,0.002384,0.174098,0.173978
6,"March 18th, 2026","April 20th, 2026",0.091667,0.435616,0.999070,200,0.002644,0.207574,0.207381
7,"April 20th, 2026","May 18th, 2026",0.077778,0.512329,0.998844,200,0.002905,0.178148,0.177942
8,"May 18th, 2026","June 18th, 2026",0.086111,0.597260,0.998573,200,0.003157,0.199405,0.199120
9,"June 18th, 2026","July 20th, 2026",0.088889,0.684932,0.998269,200,0.003426,0.208228,0.207867
10,"July 20th, 2026","August 18th, 2026",0.080556,0.764384,0.997992,200,0.003446,0.188871,0.188492
